# Chapter 8 - Data Wrangling: Join, Combine, and Reshape

# 8.1: Hierarchical Indexing

In [4]:
import pandas as pd
import numpy as np

data = pd.Series(np.random.randn(9), index = [['a', 'a', 'a', 'b', 'b','c','c','d','d'], [1,2,3,1,3,1,2,2,3]])
data

a  1    1.511116
   2   -0.608612
   3    0.316115
b  1   -1.649452
   3   -1.473995
c  1   -1.860558
   2   -0.052956
d  2   -1.514876
   3   -0.304450
dtype: float64

In [5]:
data.index

MultiIndex([('a', 1),
            ('a', 2),
            ('a', 3),
            ('b', 1),
            ('b', 3),
            ('c', 1),
            ('c', 2),
            ('d', 2),
            ('d', 3)],
           )

In [6]:
data['b']

1   -1.649452
3   -1.473995
dtype: float64

In [7]:
data['b':'c']

b  1   -1.649452
   3   -1.473995
c  1   -1.860558
   2   -0.052956
dtype: float64

In [8]:
data.loc[['b','d']]

b  1   -1.649452
   3   -1.473995
d  2   -1.514876
   3   -0.304450
dtype: float64

In [9]:
data.loc[:,2]

a   -0.608612
c   -0.052956
d   -1.514876
dtype: float64

In [10]:
data.unstack()

1         2         3
a  1.511116 -0.608612  0.316115
b -1.649452       NaN -1.473995
c -1.860558 -0.052956       NaN
d       NaN -1.514876 -0.304450

In [11]:
data.unstack().stack()

a  1    1.511116
   2   -0.608612
   3    0.316115
b  1   -1.649452
   3   -1.473995
c  1   -1.860558
   2   -0.052956
d  2   -1.514876
   3   -0.304450
dtype: float64

In [12]:
frame = pd.DataFrame(np.arange(12).reshape((4,3)), index = [['a','a','b','b'], [1,2,1,2]], columns = [['Ohio', 'Ohio', 'Colorado'], ['Green', 'Red', 'Green']])
frame

Ohio     Colorado
    Green Red    Green
a 1     0   1        2
  2     3   4        5
b 1     6   7        8
  2     9  10       11

In [13]:
frame.index.names= ['key1', 'key2']
frame.columns.names = ['state', 'color']
frame

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
     2        3   4        5
b    1        6   7        8
     2        9  10       11

### Reordering and Sorting Levels

In [14]:
frame.swaplevel('key1', 'key2')

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
2    a        3   4        5
1    b        6   7        8
2    b        9  10       11

In [15]:
frame.sort_index(level=1)

state      Ohio     Colorado
color     Green Red    Green
key1 key2                   
a    1        0   1        2
b    1        6   7        8
a    2        3   4        5
b    2        9  10       11

In [16]:
frame.swaplevel(0,1).sort_index(level=0)

state      Ohio     Colorado
color     Green Red    Green
key2 key1                   
1    a        0   1        2
     b        6   7        8
2    a        3   4        5
     b        9  10       11

### Summary Statistics

In [17]:
frame.sum(level = 'key2')

state  Ohio     Colorado
color Green Red    Green
key2                    
1         6   8       10
2        12  14       16

In [18]:
frame.sum(level= 'color', axis = 1)

color      Green  Red
key1 key2            
a    1         2    1
     2         8    4
b    1        14    7
     2        20   10

### Indexing with a DataFrame's Column

In [21]:
frame = pd.DataFrame({'a': range(7), 'b': range(7,0,-1), 'c':['one', 'one', 'one', 'two', 'two', 'two', 'two'],
                     'd': [0,1,2,0,1,2,3]})
frame

a  b    c  d
0  0  7  one  0
1  1  6  one  1
2  2  5  one  2
3  3  4  two  0
4  4  3  two  1
5  5  2  two  2
6  6  1  two  3

In [22]:
frame2 = frame.set_index(['c', 'd'])
frame2

a  b
c   d      
one 0  0  7
    1  1  6
    2  2  5
two 0  3  4
    1  4  3
    2  5  2
    3  6  1

In [23]:
frame.set_index(['c', 'd'], drop = False)

a  b    c  d
c   d              
one 0  0  7  one  0
    1  1  6  one  1
    2  2  5  one  2
two 0  3  4  two  0
    1  4  3  two  1
    2  5  2  two  2
    3  6  1  two  3

In [24]:
frame2.reset_index()

c  d  a  b
0  one  0  0  7
1  one  1  1  6
2  one  2  2  5
3  two  0  3  4
4  two  1  4  3
5  two  2  5  2
6  two  3  6  1

# 8.2 - Combining and Merging Datasets

### Database-Style DataFrame Joins

In [27]:
df1 = pd.DataFrame({'key': ['b', 'b', 'a','c', 'a','a', 'b'], 'data1' : range(7)})
df2 = pd.DataFrame({'key': ['a','b','d'], 'data2': range(3)})

In [28]:
df1

key  data1
0   b      0
1   b      1
2   a      2
3   c      3
4   a      4
5   a      5
6   b      6

In [29]:
df2

key  data2
0   a      0
1   b      1
2   d      2

In [30]:
pd.merge(df1, df2)

key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0

In [31]:
pd.merge(df1, df2, on='key')

key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0

In [32]:
df3 = pd.DataFrame({'lkey' : ['b', 'b', 'a', 'c', 'a', 'a','b'], 'data1': range(7)})

In [33]:
df4 = pd.DataFrame({'rkey': ['a', 'b', 'd'], 'data2':range(3)})

In [35]:
pd.merge(df3, df4, left_on = 'lkey', right_on = 'rkey')

lkey  data1 rkey  data2
0    b      0    b      1
1    b      1    b      1
2    b      6    b      1
3    a      2    a      0
4    a      4    a      0
5    a      5    a      0

In [36]:
pd.merge(df1, df2, how = 'outer')

key  data1  data2
0   b    0.0    1.0
1   b    1.0    1.0
2   b    6.0    1.0
3   a    2.0    0.0
4   a    4.0    0.0
5   a    5.0    0.0
6   c    3.0    NaN
7   d    NaN    2.0

In [37]:
pd.merge(df1, df2, on = 'key', how = 'left')

key  data1  data2
0   b      0    1.0
1   b      1    1.0
2   a      2    0.0
3   c      3    NaN
4   a      4    0.0
5   a      5    0.0
6   b      6    1.0

In [38]:
pd.merge(df1, df2, how='inner')

key  data1  data2
0   b      0      1
1   b      1      1
2   b      6      1
3   a      2      0
4   a      4      0
5   a      5      0

### Merging on Index

In [39]:
left1 = pd.DataFrame({'Key': ['a', 'b', 'a', 'a', 'b', 'c'], 'value' : range(6)})
right1 = pd.DataFrame({'group_val' : [3.5, 7]}, index = ['a', 'b'])

In [40]:
left1

Key  value
0   a      0
1   b      1
2   a      2
3   a      3
4   b      4
5   c      5

In [41]:
right1

group_val
a        3.5
b        7.0

In [43]:
pd.merge(left1, right1, left_on = 'Key', right_index = True)

Key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0

In [44]:
pd.merge(left1, right1, left_on = 'Key', right_index = True, how = 'outer')

Key  value  group_val
0   a      0        3.5
2   a      2        3.5
3   a      3        3.5
1   b      1        7.0
4   b      4        7.0
5   c      5        NaN

In [46]:
lefth = pd.DataFrame({'key': ['Ohio', 'Ohio', 'Ohio', 'Nevada', 'Nevada'], 'key2' : [2000, 2001,2002,2001,2002],
                     'data': np.arange(5.)})
righth = pd.DataFrame(np.arange(12).reshape((6,2)), index = [['Nevada', 'Nevada', 'Ohio', 'Ohio', 'Ohio', 'Ohio'],
                                                            [2001,2000,2000,2000,2001,2002]], columns = ['event1', 'event2'])


In [47]:
lefth

key  key2  data
0    Ohio  2000   0.0
1    Ohio  2001   1.0
2    Ohio  2002   2.0
3  Nevada  2001   3.0
4  Nevada  2002   4.0

In [48]:
righth

event1  event2
Nevada 2001       0       1
       2000       2       3
Ohio   2000       4       5
       2000       6       7
       2001       8       9
       2002      10      11

In [49]:
pd.merge(lefth, righth, left_on = ['key', 'key2'], right_index = True)

key  key2  data  event1  event2
0    Ohio  2000   0.0       4       5
0    Ohio  2000   0.0       6       7
1    Ohio  2001   1.0       8       9
2    Ohio  2002   2.0      10      11
3  Nevada  2001   3.0       0       1

In [50]:
pd.merge(lefth, righth, left_on = ['key', 'key2'], right_index = True, how = 'outer')

key  key2  data  event1  event2
0    Ohio  2000   0.0     4.0     5.0
0    Ohio  2000   0.0     6.0     7.0
1    Ohio  2001   1.0     8.0     9.0
2    Ohio  2002   2.0    10.0    11.0
3  Nevada  2001   3.0     0.0     1.0
4  Nevada  2002   4.0     NaN     NaN
4  Nevada  2000   NaN     2.0     3.0

In [51]:
left2 = pd.DataFrame([[1.,2.], [3.,4.], [5.,6.]], index = ['a','c','e'], columns = ['Ohio', 'Nevada'])

In [52]:
right2 = pd.DataFrame([[7.,8.], [9., 10.], [11.,12.], [13.,14.]], index = ['b', 'c', 'd', 'e'],
                     columns = ['Missouri', 'Alabama'])

In [53]:
left2

Ohio  Nevada
a   1.0     2.0
c   3.0     4.0
e   5.0     6.0

In [54]:
right2

Missouri  Alabama
b       7.0      8.0
c       9.0     10.0
d      11.0     12.0
e      13.0     14.0

In [55]:
left2.join(right2, how = 'outer')

Ohio  Nevada  Missouri  Alabama
a   1.0     2.0       NaN      NaN
b   NaN     NaN       7.0      8.0
c   3.0     4.0       9.0     10.0
d   NaN     NaN      11.0     12.0
e   5.0     6.0      13.0     14.0

In [57]:
left1.join(right1, on = 'Key')

Key  value  group_val
0   a      0        3.5
1   b      1        7.0
2   a      2        3.5
3   a      3        3.5
4   b      4        7.0
5   c      5        NaN

### Concatenating Along an Axis

In [58]:
arr = np.arange(12).reshape((3,4))

In [59]:
np.concatenate([arr,arr], axis = 1)

array([[ 0,  1,  2,  3,  0,  1,  2,  3],
       [ 4,  5,  6,  7,  4,  5,  6,  7],
       [ 8,  9, 10, 11,  8,  9, 10, 11]])

In [60]:
s1 = pd.Series([0,1], index = ['a', 'b'])
s2 = pd.Series([2,3,4], index = ['c', 'd', 'e'])
s3 = pd.Series([5,6], index = ['f', 'g'])

In [61]:
pd.concat([s1,s2,s3])

a    0
b    1
c    2
d    3
e    4
f    5
g    6
dtype: int64

In [62]:
pd.concat([s1,s2,s3], axis = 1)

0    1    2
a  0.0  NaN  NaN
b  1.0  NaN  NaN
c  NaN  2.0  NaN
d  NaN  3.0  NaN
e  NaN  4.0  NaN
f  NaN  NaN  5.0
g  NaN  NaN  6.0

In [63]:
s4 = pd.concat([s1,s3])

In [64]:
s4

a    0
b    1
f    5
g    6
dtype: int64

In [65]:
pd.concat([s1,s4], axis =1, join='inner')

0  1
a  0  0
b  1  1

In [68]:
result = pd.concat([s1,s1,s3], keys=['one','two','three'])
result

one    a    0
       b    1
two    a    0
       b    1
three  f    5
       g    6
dtype: int64

In [69]:
result.unstack()

a    b    f    g
one    0.0  1.0  NaN  NaN
two    0.0  1.0  NaN  NaN
three  NaN  NaN  5.0  6.0

In [71]:
pd.concat([df1, df2], axis=1, keys=['level1', 'level2'])

level1       level2      
     key data1    key data2
0      b     0      a   0.0
1      b     1      b   1.0
2      a     2      d   2.0
3      c     3    NaN   NaN
4      a     4    NaN   NaN
5      a     5    NaN   NaN
6      b     6    NaN   NaN

In [72]:
pd.concat({'level1':df1, 'level2':df2}, axis = 1)

level1       level2      
     key data1    key data2
0      b     0      a   0.0
1      b     1      b   1.0
2      a     2      d   2.0
3      c     3    NaN   NaN
4      a     4    NaN   NaN
5      a     5    NaN   NaN
6      b     6    NaN   NaN

In [73]:
pd.concat([df1, df2], axis = 1, keys =['level1', 'level2'], names =['upper', 'lower'])

upper level1       level2      
lower    key data1    key data2
0          b     0      a   0.0
1          b     1      b   1.0
2          a     2      d   2.0
3          c     3    NaN   NaN
4          a     4    NaN   NaN
5          a     5    NaN   NaN
6          b     6    NaN   NaN

In [74]:
pd.concat([df1, df2], ignore_index = True)

key  data1  data2
0   b    0.0    NaN
1   b    1.0    NaN
2   a    2.0    NaN
3   c    3.0    NaN
4   a    4.0    NaN
5   a    5.0    NaN
6   b    6.0    NaN
7   a    NaN    0.0
8   b    NaN    1.0
9   d    NaN    2.0

### Combining Data with Overlap

In [76]:
a = pd.Series([np.nan, 2.5, np.nan, 3.5, 4.5, np.nan], index = ['f', 'e', 'd', 'c', 'b', 'a'])
b = pd.Series(np.arange(len(a), dtype = np.float64), index = ['f','e','d', 'c','b','a'])

In [77]:
a

f    NaN
e    2.5
d    NaN
c    3.5
b    4.5
a    NaN
dtype: float64

In [78]:
b

f    0.0
e    1.0
d    2.0
c    3.0
b    4.0
a    5.0
dtype: float64

In [79]:
b[-1]

5.0

In [80]:
np.where(pd.isnull(a),b,a)

array([0. , 2.5, 2. , 3.5, 4.5, 5. ])

In [81]:
b[:-2].combine_first(a[2:])

a    NaN
b    4.5
c    3.0
d    2.0
e    1.0
f    0.0
dtype: float64

In [82]:
df1.combine_first(df2)

data1  data2 key
0      0    0.0   b
1      1    1.0   b
2      2    2.0   a
3      3    NaN   c
4      4    NaN   a
5      5    NaN   a
6      6    NaN   b

# 8.3: Reshaping and Pivoting

### Reshaping with Hierarchical Indexing

In [84]:
data = pd.DataFrame(np.arange(6).reshape((2,3)), index = pd.Index(['Ohio', 'Colorado'], name = 'state'),
                   columns = pd.Index(['one', 'two', 'three'], name = 'number'))

In [85]:
data

number    one  two  three
state                    
Ohio        0    1      2
Colorado    3    4      5

In [87]:
result = data.stack()

In [88]:
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [89]:
result.unstack()
result

state     number
Ohio      one       0
          two       1
          three     2
Colorado  one       3
          two       4
          three     5
dtype: int64

In [90]:
result.unstack(0)

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5

In [91]:
result.unstack('state')

state   Ohio  Colorado
number                
one        0         3
two        1         4
three      2         5

In [92]:
s1 = pd.Series([0,1,2,3], index = ['a', 'b', 'c','d'])
s2 = pd.Series([4,5,6], index = ['c','d','e'])
data2 = pd.concat([s1,s2], keys=['one','two'])
data2

one  a    0
     b    1
     c    2
     d    3
two  c    4
     d    5
     e    6
dtype: int64

In [93]:
data2.unstack()

a    b    c    d    e
one  0.0  1.0  2.0  3.0  NaN
two  NaN  NaN  4.0  5.0  6.0

In [94]:
data2.unstack().stack()

one  a    0.0
     b    1.0
     c    2.0
     d    3.0
two  c    4.0
     d    5.0
     e    6.0
dtype: float64

In [95]:
df = pd.DataFrame({'left': result, 'right':result + 5}, columns = pd.Index(['left','right'], name = 'side'))
df

side             left  right
state    number             
Ohio     one        0      5
         two        1      6
         three      2      7
Colorado one        3      8
         two        4      9
         three      5     10

In [96]:
df.unstack('state').stack('side')

state         Colorado  Ohio
number side                 
one    left          3     0
       right         8     5
two    left          4     1
       right         9     6
three  left          5     2
       right        10     7

### Pivoting "Wide" to "Long" Format

In [99]:
df = pd.DataFrame({'key': ['foo', 'bar','baz'], 'A': [1,2,3], 'B': [4,5,6], 'C':[7,8,9]})
df

key  A  B  C
0  foo  1  4  7
1  bar  2  5  8
2  baz  3  6  9

In [100]:
melted = pd.melt(df,['key'])

In [102]:
melted

key variable  value
0  foo        A      1
1  bar        A      2
2  baz        A      3
3  foo        B      4
4  bar        B      5
5  baz        B      6
6  foo        C      7
7  bar        C      8
8  baz        C      9

In [103]:
reshaped = melted.pivot('key', 'variable', 'value')
reshaped

variable  A  B  C
key              
bar       2  5  8
baz       3  6  9
foo       1  4  7

In [104]:
reshaped.reset_index()

variable  key  A  B  C
0         bar  2  5  8
1         baz  3  6  9
2         foo  1  4  7

In [106]:
pd.melt(df, value_vars=['A','B'])

variable  value
0        A      1
1        A      2
2        A      3
3        B      4
4        B      5
5        B      6